In [11]:
import csv 

import numpy as np
import pandas as pd
import geopy.distance as geo_dis
import plotly.express as plt
from matplotlib.widgets import Slider
import scipy

In [12]:
def getFilenames():
    file_names = []

    for i in range(15):
        file_names.append('Year_{}.csv'.format(2001 + i))
        
    return file_names;

In [13]:
def getSnowGridLatLonMap():
    snotel_grids_info = pd.read_csv('SNOTEL_join_PMW_grids.csv')

    grid_lat_lon_map = {}

    for index, row_info in snotel_grids_info.iterrows():
        grid_lat_lon_map[row_info['PMW_lat_lon']] = row_info['Station.Name']
        
    return grid_lat_lon_map
    

In [14]:
def getCorrespondingSnotelName(lat_lon_values, lat_lon_map):
    snotel_names = []
    
    for lat_lon in lat_lon_values:
        snotel_names.append(lat_lon_map['{}'.format(lat_lon)])
    
    return snotel_names
    
def getDFData():
    file_names = getFilenames()
    
    grid_lat_lon_map = getSnowGridLatLonMap()
    
    all_year_dataframes = []

    for index, filename in enumerate(file_names):
        file = pd.read_csv(filename,header = 0)
    
        file['Snotel Location'] = getCorrespondingSnotelName(file['lat_lon'], grid_lat_lon_map)
    
        pivot_values_columns = []
        for column in file.columns:
            if (column != 'Snotel Location'  and column != 'x' and column != 'y' and column != 'lat_lon'):
                pivot_values_columns.append(column)
    
    
        pivoted_table = pd.pivot_table(file, columns=['Snotel Location'], values = pivot_values_columns)
    
        all_year_dataframes.append(pivoted_table)
    
    return pd.concat(all_year_dataframes, ignore_index=True)



In [16]:
dfData = getDFData()

compression_opts = dict(method='zip',
                        archive_name='df_reading.csv')  
dfData.to_csv('out.zip', index=False,
          compression=compression_opts)  